In [ ]:
import h5py
from astropy.io import ascii
import numpy as np
import pandas as pd
import os
import csv
from seaborn import histplot, pointplot, lineplot
import scipy
import matplotlib.pyplot as plt
from scipy.stats import crystalball
from matplotlib.transforms import Affine2D
import random
import copy
import plotly.graph_objects as go
from os import path, makedirs
from astropy.coordinates import SkyCoord
from astropy.table import Table
import astropy.units as u
from ctapipe.containers import MuonEfficiencyContainer
from ctapipe.coordinates import CameraFrame, TelescopeFrame
from ctapipe.io import EventSource, EventSeeker
from ctapipe.core import TelescopeComponent
from ctapipe.utils.datasets import get_dataset_path
from ctapipe.core.traits import (
    List,
    Int,
    FloatTelescopeParameter,
    TelescopeParameter,
    Unicode,
)
from ctapipe.image.cleaning import tailcuts_clean
from ctapipe.image.muon import MuonRingFitter, MuonIntensityFitter
from traitlets.config import Config
from ctapipe.io import EventSource
from ctapipe.image import ImageProcessor
from ctapipe.image.muon import MuonProcessor
import matplotlib.pyplot as plt
from ctapipe.utils import get_dataset_path
from ctapipe.io import EventSource
from ctapipe.visualization import CameraDisplay
from ctapipe.instrument import CameraGeometry
from ctapipe.calib import CameraCalibrator
import ctapipe
from eventio import EventIOFile
from eventio.search_utils import yield_all_objects_depth_first
from eventio import SimTelFile

In [ ]:
filename = "/Users/vdk/Downloads/events_after_muon_tagging_all.pog.h5"

with h5py.File(filename, "r") as f:
    pass

In [ ]:
f = h5py.File('/Users/vdk/Downloads/events_after_muon_tagging_all.pog.h5', 'r')

In [ ]:
filename = "/Users/vdk/MST_work/data_muons/events_after_muon_tagging_all.pog.h5"
#filename = '/Users/vdk/Downloads/events_after_muon_tagging_all.pog.h5'
source = EventSource(input_url = filename, max_events=5)
#source = ctapipe.io.SimTelEventSource(filename)
#source = ctapipe.io.HDF5EventSource(filename)
#source = ctapipe.io.TableLoader(filename)


In [ ]:
os.access(filename, os.R_OK)

In [ ]:
import scipy.io
f = scipy.io.loadmat('/Users/vdk/MST_work/data_muons/events_after_muon_tagging_all_opt_deg_random.pog.h5')

In [ ]:
path = "/Users/vdk/MST_work/data_muons/events_after_muon_tagging_all.pog.h5"

with EventIOFile(path) as f:
    for o, level in yield_all_objects_depth_first(f):
        if hasattr(o, 'parse'):
            o.parse()

In [ ]:
test = SimTelFile(filename)

In [ ]:
import numpy as np
import pandas as pd
import argparse, os, sys, subprocess, h5py
import astropy.units as u
from astropy.table import Table, unique, vstack, QTable, hstack, join, Column
from eventio import SimTelFile

parser = argparse.ArgumentParser(description="Specify simulation file")

parser.add_argument('simtel_urls',nargs='+',default=[],help='Enter sim_telarray file')
parser.add_argument('run_no',nargs='+',default=0,help='Enter run number')

args = parser.parse_args()
dicts = vars(args)

simtel_urls = dicts['simtel_urls']
run_n = int(dicts["run_no"][0])
print(run_n)
patch_url = "/lfs/l1/cta/nieves/muon-tagging/fc1764_layout_patches.txt"  #contains position of the patches and cards

def get_events(simtel_urls, patch_url, run_n):
    """
    simtel_url : sim_telarray simulation
    patch_url : file with positions of patches and cards in the camera
    """
    X = pd.read_csv(patch_url, sep=" ", index_col=None)
    trigger_id = X["trigger_addr"]

    no_trig_sect = []
    no_trig_tel = []
    no_trig_card = []
    event_id = []
    tel_id = []
    shower_id = []
    r0_traces = []
    r1_traces = []
    dc_to_pe = []
    pedestal = []
    for simtel_url in simtel_urls:
        with SimTelFile(simtel_url) as f:   #reading trigger & event information with eventio
            for e in f:
                tels = e['telescope_events'].keys()
                for tel in tels:
                    t_trig = e['telescope_events'][tel]['header']['time_trgsect']
                    trig_sects = e['telescope_events'][tel]['header']['list_trgsect'][t_trig < (np.min(t_trig) + 8)]
                    no_trig_sect.append(len(trig_sects)) #no of triggered sectors

                    trig_card = []
                    for string in np.array(trigger_id.values)[np.array(trig_sects)]:
                        trig_card.append(string[2])

                    no_trig_card.append(len(np.unique(trig_card)))  #no of triggered cards
                    no_trig_tel.append(len(e["trigger_information"]["triggered_telescopes"]))  #no of triggered tels to study background
                    event_id.append(e.get("event_id", 0))
                    tel_id.append(tel)
                    pedestal.append(e["camera_monitorings"][tel]["pedestal"][0] / e["camera_monitorings"][tel]["n_ped_slices"])
                    dc_to_pe.append(e["laser_calibrations"][tel]["calib"][0])
                    shower_id.append(e["mc_event"]["shower_num"])
                    r0_traces.append(e["telescope_events"][tel]["adc_samples"][0])
                    r1_traces.append((e["telescope_events"][tel]["adc_samples"][0] - (e["camera_monitorings"][tel]["pedestal"][0][:,None] / e["camera_monitorings"][tel]["n_ped_slices"]))*e["laser_calibrations"][tel]["calib"][0][:,None])
                
    cut_p = np.array(no_trig_sect) >= 30  #patch cut
    cut_c = np.array(no_trig_card) >= 3   #card cut

    events_after_tagging = QTable([np.array(event_id)[(cut_p) & (cut_c)], np.array(shower_id)[(cut_p) & (cut_c)], np.array(tel_id)[(cut_p) & (cut_c)], np.array(no_trig_tel)[(cut_p) & (cut_c)], np.array(dc_to_pe)[(cut_p) & (cut_c)], np.array(pedestal)[(cut_p) & (cut_c)], np.array(r1_traces)[(cut_p) & (cut_c)], np.array(r0_traces)[(cut_p) & (cut_c)]], names=("Event_ID", "Shower_Number", "Telescope_Num", "Num_Trigger_Tel", "dc_to_pe", "pedestal", "R1_Traces", "R0_Traces"))

    events_after_tagging.write('/lfs/l1/cta/nieves/muon-tagging/dataset/results/events_after_muon_tagging_{}.pog.h5'.format(run_n), format='fits', overwrite=True) 

get_events(simtel_urls, patch_url, run_n)


In [ ]:
t = Table.read("/Users/vdk/MST_work/data_muons/events_after_muon_tagging_all.pog.h5")

In [ ]:
t

In [ ]:
np.shape(t['R0_Traces'])

In [ ]:
t['R1_Traces'][3]

In [ ]:
t2 = Table.read("/Users/vdk/MST_work/data_muons/events_after_muon_tagging_all_opt_deg_random.pog.h5")

In [ ]:
t2